# Code to inference model to recoginize different hand signs as letters.

In [6]:
import cv2
import os
import mediapipe as mp
import numpy as np
from collections import defaultdict
import torch

In [9]:
DEVICE = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {DEVICE} device")

Using cuda device


In [12]:
# Load model for inferencing
model = torch.jit.load('model.pth')
model.eval()
model.to(DEVICE)

RecursiveScriptModule(
  original_name=NeuralNetwork
  (net): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Linear)
    (3): RecursiveScriptModule(original_name=LogSoftmax)
  )
)

In [11]:
cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.6, max_num_hands = 1)

In [ ]:
dict_ = defaultdict(int)
while True:
    data_aux, x_, y_ = [], [], []
    
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    H, W, channel = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    #Only predicting left hand
    if results.multi_hand_landmarks and results.multi_handedness[0].classification[0].label == 'Left':
        for hand_landmarks in results.multi_hand_landmarks:
            # Drawing landmarks per displayed frame
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y

            x_.append(x)
            y_.append(y)

        x_min, x_max = min(x_), max(x_)
        y_min, y_max = min(y_), max(y_)
        
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            data_aux.append(x - x_min)
            data_aux.append(y - y_min)

        # (x1, y1) is the top left corner of the box
        x1 = int(x_min * W) - 10
        y1 = int(y_min * H) - 10

        # (x2, y2) is the bottom right corner of the box
        x2 = int(x_max * W) - 10
        y2 = int(y_max * H) - 10

        predictions = model.predict_proba([np.asarray(data_aux)])
        if max(predictions[0]) >= 0.30:
            prediction = model.predict([np.asarray(data_aux)])
            predicted_character = chr(int(prediction[0]) + 65) #chr(65) = 'A'
            dict_[predicted_character] += 1
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
    else:
        if dict_ and max(dict_.values()) >= 30:
            WORD += max(dict_, key=dict_.get)
            dict_ = defaultdict(int)
        
    cv2.imshow('frame', frame)
    print(WORD)
    # Press 'q' to quit program
    if cv2.waitKey(1) == ord('q'): break

cap.release()
cv2.destroyAllWindows()